In [1]:
import numpy as np
import torch
import torchvision as tv

import os
import pickle
import time
import itertools

import PIL
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [7]:
logpath = 'failure_log.pth'
faillog = torch.load(logpath)
fail_ids_t = faillog['fail_ids_t']
fail_ids_i = faillog['fail_ids_i']
print len(fail_ids_t)

1769
